# chapter 4. Classification

## Bayesian Optimization

### 베이지안 최적화 개요

**베이지안 최적화**는 목적 함수 식을 제대로 알 수 없는 블랙 박스 형태의 함수에서, 최대 또는 최소 함수 반환 값을 만드는 최적 입력값을 가능한 적은 시도를 통해 빠르고 효과적으로 찾아주는 방법이다. <br>
**베이지안 확률에 기반**을 두고 있는 최적화 기법으로, **새로운 데이터를 입력받았을 때 최적 함수를 예측하는 사후 모델을 개선해나가면서 최적 함수 모델을 만들어 낸다.** <br>
베이지안 최적화를 구성하는 두 가지 중요 요소는 **대체 모델(Surrogate Model)** 과 **획득 함수(Acquisition Function)** 이다. <br>
대체 모델은 획득 함수로부터 최적 함수를 예측할 수 있는 입력값을 추천 받은 뒤 이를 기반으로 최적 함수 모델을 개선해 나가며, 획득 함수는 개선된 대체 모델을 기반으로 최적 입력값을 계산한다. <br>

**베이지안 최적화 단계** <br>
1. 랜덤하게 하이퍼 파라미터를 샘플링, 성능 결과를 관측한다. <br>
2. 관측된 값을 기반으로 대체 모델은 최적 함수를 추정한다. <br>
3. 추정된 최적 함수를 기반으로 획득 함수는 다음으로 관측할 하이퍼 파라미터 값을 계산한다. <br>
4. 획득 함수로부터 전달된 하이퍼 파라미터를 수행하여 관측된 값을 기반으로 대체 모델은 갱신되어 다시 최적 함수를 예측 추정한다. <br>

대체 모델은 최적 함수를 추정할 때 일반적으로 가우시안 프로세스(Gaussian Process)를 적용한다. 

### HyperOpt

베이지안 최적화를 머신러닝 모델의 하이퍼 파라미터 튜닝에 적용할 수 있게 제공되는 여러 파이썬 패키지들이 있는데, 대표적으로 HyperOpt, Bayesian Optimization, Optuna 등이 있다. <br>

**HyperOpt를 활용하는 주요 로직 구성** <br>
1. 입력 변수명, 입력값의 검색 공간을 설정한다. <br>
2. 목적 함수의 설정 <br>
3. 목적 함수의 반환 최소값을 가지는 최적 입력값을 유추한다. <br>

**HyperOpt를 사용할 때 유의할 점은 다른 패키지와 다르게 목적 함수 반환 값이 최대값이 아닌 최소값을 가지는 최적 입력값을 유추한다는 것이다.** 

In [2]:
from hyperopt import hp

# -10 ~ 10까지 1 간격을 가지는 입력 변수 x와 -15 ~ 15까지 1 간격으로 입력 변수 y를 설정
search_space = {'x' : hp.quniform('x', -10, 10, 1), 'y' : hp.quniform('y', -15, 15, 1)}

**입력값의 검색 공간을 제공하는 대표적 함수**

|함수|내용|
|:------|:---|
|hp.quniform(label, low, high, q)|lable로 지정된 입력값 변수 검색 공간을 최소값 low에서 최대값 high까지 q의 간격을 가지고 설정한다. |
|hp.uniform(label, low, high)|최소값 low에서 최대값 high까지 정규 분포 형태의 검색 공간 설정 |
|hp.randint(label, low, high)|exp(uniform(low, high)) 값을 반환하며 반환 값의 log 변환된 값은 정규 분포 형태를 가지는 검색 공간 설정|
|hp.choice(label, options)|검색 값이 문자열 또는 문자열과 숫자값이 섞여 있을 경우 설정한다. |

목적 함수는 반드시 변수값과 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환하는 구조로 만들어져야 한다. 

In [3]:
from hyperopt import STATUS_OK

# 목적 함수를 생성, 변수값과 변수 검색 공간을 가지는 딕셔너리를 인자로 받고 특정 값을 반환한다.
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    
    retval = x**2 - 20*y
    
    return retval

목적 함수의 반환값이 최소가 될 수 있는 최적의 입력값을 베이지안 최적화 기법에 기반하여 찾아야 한다. <br>
HyperOpt는 fmin(objective, space, algo, max_evals, trials)함수를 제공한다. <br>

**fmin() 주요 인자**

|함수|내용|
|:------|:---|
|fn|목적 함수|
|space|검색 공간 딕셔너리|
|algo|베이지안 최적화 적용 알고리즘, 기본적으로 tpe.suggest이며 이는 HyperOpt의 기본 최적화 알고리즘인 TPE(Tree of Parzen Estimator)를 의미한다. |
|max_evals|최적 입력값을 찾기 위한 입력값 시도 횟수이다. |
|trials|최적 입력값을 찾기 위해 시도한 입력값 및 해당 입력값의 목적 함수 반환값 결과를 저장하는데 사용한다. |
|rstate|fmin()을 수행할 때마다 동일한 결과값을 가질 수 있도록 설정하는 랜덤 시드 값이다. |

In [5]:
import numpy as np
from hyperopt import fmin, tpe, Trials

# 입력 결과값을 저장한 Trials 객체값 생성
trial_val = Trials()

# 목적 함수의 최소값을 반환하는 최적 입력 변수값을 5번의 입력값 시도(max_evals = 5)로 찾아냈다.
best_01 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 5, trials = trial_val, rstate = np.random.default_rng(seed = 0))

print('best : ', best_01)

100%|██████████████████████| 5/5 [00:00<00:00, 214.12trial/s, best loss: -224.0]
best :  {'x': -4.0, 'y': 12.0}


In [6]:
# 입력 결과값을 저장한 Trials 객체값 생성
trial_val = Trials()

# max_evals = 20로 변경해 다시 테스트
best_02 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 20, trials = trial_val, rstate = np.random.default_rng(seed = 0))

print('best : ', best_02)

100%|████████████████████| 20/20 [00:00<00:00, 533.60trial/s, best loss: -296.0]
best :  {'x': 2.0, 'y': 15.0}


fmin() 함수 수행 시 인자로 들어가는 Trials 객체는 함수의 반복 수행시마다 입력되는 변수값들과 함수 반환값을 속성으로 가지고 있다. <br>
**Trials 객체의 중요 속성**은 **results와 vals**가 있다. 

In [7]:
# fmin()에 인자로 들어가는 Trials 객체의 result 속성에 파이썬 리스트로 목적 함수 반환값들이 저장된다.
# 리스트 내부의 개별 원소는 {'loss' : 함수 반환값, 'status' : 반환 상태값}과 같은 딕셔너리이다.
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


In [8]:
# Trials 객체의 vals 속성에 {'입력 변수명' : 개별 수행 시마다 입력된 값 리스트} 형태로 저장된다.
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


vals는 딕셔너리 형태의 값을 가지며, 입력 변수 x와 y를 키 값으로 가지며, x와 y 키 값의 value는 20회의 반복 수행시마다 사용되는 입력값들을 리스트 형태로 가지고 있는 것을 알 수 있다. <br>

**Trials 객체의 results와 vals 속성**은 **HyperOpt의 fmin() 함수의 수행 시마다 최적화되는 경과를 볼 수 있는 함수 반환값**과 **입력 변수값들의 정보**를 제공한다. 

In [9]:
import pandas as pd

# results에서 loss 키 값에 해당하는 values를 추출해 리스트로 생성
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성
result_df = pd.DataFrame({'x' : trial_val.vals['x'], 'y' : trial_val.vals['y'], 'losses' : losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0
